## Writing vectorized code in NumPy

Let $A$ and $B$ be the matrices representing points in $d$ dimensional space, each having $m$ and $n$ points (rows) respectively, i.e., $A$ is $m \times d$ and $B$ is $n \times d$.

Let the task be to calculate the Euclidean distance for each point from $A$ to each point in $B$, the result is $m \times $n matrix $C$, where $C_{i, j}$ represents distance between $i^{th}$ point of $A$ and $j^{th}$ point of $B$.

In [1]:
import numpy as np

# Dimensions
D = 500
M = 700
N = 1000

# matrices
A = np.random.rand(M, D)
B = np.random.rand(N, D)

### Naive implementation

Naive implementation of this task will be going for each point in $A$ and for each point in $B$ and calculating the corresponding euclidean distances.

In [2]:
def distances_naive(A, B):
    """Compute Euclidean distance from each point in A to each point in B.
    Args: 
        A: `numpy.ndarray` MxD dimensional array.
        B: `numpy.ndarray` NxD dimensional.
    
    Returns:
        C: `numpy.ndarray` distances matrix.
        
    Raises:
        ValueError: Incompatible matrix dimensions.
    """
    
    if A.shape[1] != B.shape[1]:
        raise ValueError('Incompatible dimensions of matrices!')
    
    C = np.empty((A.shape[0], B.shape[0]))
    for i in range(A.shape[0]):
        for j in range(B.shape[0]):
            C[i, j] = np.sqrt(np.sum(np.square(A[i] - B[j])))
    
    return C

Let's write function for measuring execution time of any given function:

In [3]:
def print_exec_time(f, name='This function', *args):
    """Print execution time for the given function.
    Args:
        f: function to execute.
        name: `string` name of the function or any comment.
        *args: arguments to pass to function.
    
    Returns:
        Return value of function.
    """
    
    import time
    tic = time.time()
    ret = f(*args)
    toc = time.time()
    print("%s took %f seconds."%(name, toc-tic))
    return ret

In [4]:
C1 = print_exec_time(distances_naive, 'Naive implementation', A, B)

Naive implementation took 4.768615 seconds.


### Vectorized Implementation

Now, let's try to accomplish the same task with different approach. Instead of explicitly performing Python `for` loops we will use basic Linear Algebra and NumPy Broadcasting. There is a very good tutorial on NumPy Broadcasting by <a href="https://eli.thegreenplace.net/2015/broadcasting-arrays-in-numpy/">Eli Bendersky.</a>

In [5]:
def distances_vectorized(A, B):
    """Compute Euclidean distance from each point in A to each point in B.
    
    Args: 
        A: `numpy.ndarray` MxD dimensional array.
        B: `numpy.ndarray` NxD dimensional.
    Returns:
        C: `numpy.ndarray` distances matrix
    
    Raises:
        ValueError: Incompatible matrix dimensions.
    """
    
    if A.shape[1] != B.shape[1]:
        raise ValueError('Incompatible dimensions of matrices!')
        
    # (A - B)^2 = A^2 - 2AB + B^2
    A_sqr = np.sum(np.square(A), axis=1).reshape(A.shape[0], -1)
    B_sqr = np.sum(np.square(B), axis=1)
    C = np.sqrt(A_sqr - 2*A.dot(B.T) + B_sqr)
    return C

In [6]:
C2 = print_exec_time(distances_vectorized, 'Vectorized implementation', A, B)

Vectorized implementation took 0.100518 seconds.


As you can see vectorized implementation significantly outperforms naive implementation. The main reason for this is that in vectorization `for` loops are performed internally by NumPy in fast C code instead of slow Python code.

Before going further in details of vectorized implementation, let's check whether naive implementation agrees with vectorized implementation. For checking this we will use Frobenius norm of the difference matrix $C1$ and $C2$.

In [9]:
def compare_matrices(A, B, eps=0.001):
    """Compare whether two matrices are identical.
    
    Args: 
        A: `numpy.ndarray` MxD dimensional array.
        B: `numpy.ndarray` NxD dimensional.
    
    Returns: `boolean`
    
    Raises:
        ValueError: Incompatible matrix dimensions.
    """
    if A.shape[1] != B.shape[1]:
        raise ValueError('Incompatible dimensions of matrices!')
        
    diff = np.linalg.norm(A - B, ord='fro')
    print("Difference: %f" % diff)
    return diff < eps

In [11]:
compare_matrices(C1, C2)

Difference: 0.000000


True

### Mathematical explanation

There are only 3 lines of code in vectorized implementation. But we will go through the details step by step, clarifying every aspect of the code.

Let $A = \begin{bmatrix} 
    a_{11} & a_{12} & a_{13} & \dots & a_{1d} \\
    a_{21} & a_{22} & a_{23} & \dots & a_{2d} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    a_{m1} & a_{m2} & a_{m3} & \dots & a_{md}
\end{bmatrix}$, 
$B = \begin{bmatrix} 
    b_{11} & b_{12} & b_{13} & \dots & b_{1d} \\
    b_{21} & b_{22} & b_{23} & \dots & b_{2d} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    b_{n1} & b_{n2} & b_{n3} & \dots & b_{nd}
\end{bmatrix}$

What we really want at the end is matrix $C = \begin{bmatrix} 
    c_{11} & c_{12} & c_{13} & \dots & c_{1n} \\
    c_{21} & c_{22} & c_{23} & \dots & c_{2n} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    c_{m1} & c_{m2} & c_{m3} & \dots & c_{mn}
\end{bmatrix}$

Where $c_{ij} = \sqrt{(a_{i1} - b_{j1})^2 + (a_{i2} - b_{j2})^2 + \dots + (a_{id} - b_{jd})^2}$ Euclidean distance between point (row) $i$ in $A$ and point (row) $j$ in $B$.

Expanding the expression under the square root gives the following:

$(a_{i1} - b_{j1})^2 + (a_{i2} - b_{j2})^2 + \dots + (a_{id} - b_{jd})^2 = 
a_{i1}^2 + \dots + a_{id}^2 + b_{j1}^2 + \dots + b_{jd}^2 - 2(a_{i1}b_{j1} + \dots + a_{id}b_{jd})$

The **crucial observation** here is that sum $a_{i1}^2 + \dots + a_{id}^2$ appears in every column of row $i$ of $C$, while sum $b_{j1}^2 + \dots + b_{jd}^2$ appears in every row of column $j$ of $C$.

The sum $a_{i1}^2 + \dots + a_{id}^2$ is just summing the square of each entry in $i^{th}$ row of matrix $A$. This is exactly what code snippet in vectorized implementation `A_sqr = np.sum(np.square(A), axis=1).reshape(A.shape[0], -1)` does for all rows in $A$. More concretely, `A_sqr` will be NumPy array of shape $(m, 1)$ where each element of `A_sqr[m, i]` is sum of squares of entries in $i^{th}$ row of $A$.  Here, we reshape `A_sqr`, because we want it in every column of $C$.

Similar argument is true for $b_{j1}^2 + \dots + b_{jd}^2$ and line `B_sqr = np.sum(np.square(B), axis=1)` does the same for all rows in B. Notice, that we do not reshape this, because we want it in every row of $C$.

The sum $(a_{i1}b_{j1} + \dots + a_{id}b_{jd})$ can be written as $ab^{T}$, where $a$ and $b$ are treated as **row vectors**. For all rows in $A$ and for all rows in $B$ this can be written as $AB^{T}$.

The line `C = np.sqrt(A_sqr - 2*A.dot(B.T) + B_sqr)` gives the final result, but do not forget to take the square root!